In [21]:
import sys, subprocess, io, warnings
warnings.filterwarnings('ignore')

def install_packages():
    packages = ["ccxt", "pandas", "bokeh", "requests", "numpy", "ipywidgets"]
    for package in packages:
        try:
            __import__(package)
        except ImportError:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

old_stdout = sys.stdout
sys.stdout = io.StringIO()
install_packages()
sys.stdout = old_stdout

import ccxt
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta
import time
from bokeh.plotting import figure, show, output_notebook, save
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, HoverTool, CrosshairTool, Range1d, BoxAnnotation
from bokeh.models import NumeralTickFormatter, DatetimeTickFormatter, Button, CustomJS
from bokeh.io import output_file
from IPython.display import HTML, display, clear_output

output_notebook(hide_banner=True)

def get_data_silently():
    def get_fear_greed_history():
        url = "https://api.alternative.me/fng/?limit=0"
        try:
            response = requests.get(url)
            data = response.json()
            if data['metadata']['error'] is None:
                df = pd.DataFrame(data['data'])
                df['value'] = df['value'].astype(float)
                df['timestamp'] = pd.to_datetime(df['timestamp'].astype(int), unit='s')
                df['date'] = df['timestamp'].dt.date
                df = df.sort_values('timestamp')
                return df
            else:
                return None
        except:
            return None

    fear_greed_df = get_fear_greed_history()
    if fear_greed_df is None:
        return None

    exchange = ccxt.binance({'enableRateLimit': True})

    def get_binance_historical_data(symbol, timeframe):
        if fear_greed_df is not None:
            start_date = fear_greed_df['timestamp'].min().to_pydatetime()
            since = int(start_date.timestamp() * 1000)
            all_candles = []
            
            while True:
                try:
                    candles = exchange.fetch_ohlcv(symbol, timeframe, since, limit=1000)
                    if not candles or len(candles) == 0:
                        break
                    all_candles.extend(candles)
                    since = candles[-1][0] + 1
                    time.sleep(exchange.rateLimit / 1000)
                    if since > int(datetime.now().timestamp() * 1000):
                        break
                except:
                    time.sleep(10)
                    continue
            
            df = pd.DataFrame(all_candles, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            df['date'] = df['timestamp'].dt.date
            return df
        return None

    btc_df = get_binance_historical_data('BTC/USDT', '1d')
    if btc_df is None:
        return None

    if fear_greed_df is not None and btc_df is not None:
        fear_greed_df['date_str'] = fear_greed_df['date'].astype(str)
        btc_df['date_str'] = btc_df['date'].astype(str)
        merged_df = pd.merge(btc_df, fear_greed_df[['date_str', 'value', 'value_classification']], on='date_str', how='left')
        
        # Calculer le pourcentage de fear
        merged_df['fear_percent'] = 100 - merged_df['value']
        
        # Calculer les moyennes mobiles pour Fear & Greed avec min_periods pour ignorer les trous
        merged_df['ma31'] = merged_df['value'].rolling(window=31, min_periods=30).mean()
        merged_df['ma62'] = merged_df['value'].rolling(window=62, min_periods=60).mean()
        merged_df['ma90'] = merged_df['value'].rolling(window=90, min_periods=88).mean()
        merged_df['ma365'] = merged_df['value'].rolling(window=365, min_periods=360).mean()
        
        return merged_df
    return None

def create_ohlc_fear_greed_bokeh(data, title, period_name=""):
    inc = data.close > data.open
    dec = data.close <= data.open
    
    source_inc = ColumnDataSource(data=data[inc])
    source_dec = ColumnDataSource(data=data[dec])
    source_all = ColumnDataSource(data=data)
    
    w = 12*60*60*1000
    
    y_min = data['low'].min() * 0.98
    y_max = data['high'].max() * 1.02
    
    p_ohlc = figure(
        title=f'BTC/USDT OHLC {period_name}',
        x_axis_type='datetime',
        sizing_mode="stretch_width",
        height=500,
        tools="pan,wheel_zoom,box_zoom,reset,save",
        toolbar_location="above",
        background_fill_color="#1f1f1f",
        border_fill_color="#1f1f1f",
        outline_line_color=None,
        min_border_bottom=0,
        margin=(10, 10, 0, 10),
        y_range=(y_min, y_max)
    )
    
    p_ohlc.segment(x0='timestamp', y0='low', x1='timestamp', y1='high', color="#00ff00", source=source_inc)
    p_ohlc.vbar(x='timestamp', width=w, top='close', bottom='open', fill_color="#00ff00", line_color="#00ff00", source=source_inc)
    
    p_ohlc.segment(x0='timestamp', y0='low', x1='timestamp', y1='high', color="#ff0000", source=source_dec)
    p_ohlc.vbar(x='timestamp', width=w, top='open', bottom='close', fill_color="#ff0000", line_color="#ff0000", source=source_dec)
    
    p_ohlc.yaxis.formatter = NumeralTickFormatter(format="0,0")
    
    p_ohlc.xgrid.grid_line_color = None
    p_ohlc.ygrid.grid_line_color = None
    
    p_ohlc.xaxis.axis_line_color = "#555555"
    p_ohlc.yaxis.axis_line_color = "#555555"
    p_ohlc.xaxis.major_label_text_color = "#cccccc"
    p_ohlc.yaxis.major_label_text_color = "#cccccc"
    p_ohlc.title.text_color = "#ffffff"
    
    hover_ohlc = HoverTool(
        tooltips="""
        <div style="background-color: rgba(42, 42, 42, 0.7); color: white; padding: 5px; border-radius: 3px; font-size: 11px;">
            <div>@timestamp{%F}</div>
            <div>O: @open{0,0.00}</div>
            <div>C: @close{0,0.00}</div>
        </div>
        """,
        formatters={'@timestamp': 'datetime'},
        mode='vline'
    )
    p_ohlc.add_tools(hover_ohlc)
    
    p_fg = figure(
        title="Indice Fear & Greed",
        x_axis_type='datetime',
        sizing_mode="stretch_width",
        height=300,
        x_range=p_ohlc.x_range,
        tools="pan,wheel_zoom,box_zoom,reset,save",
        toolbar_location="above",
        background_fill_color="#1f1f1f",
        border_fill_color="#1f1f1f",
        outline_line_color=None,
        min_border_bottom=10,
        margin=(0, 10, 10, 10)
    )
    
    # Créer la ligne Fear & Greed
    fg_line = p_fg.line(x='timestamp', y='value', source=source_all, line_width=2, color='#3a7ca8', legend_label='Fear & Greed')
    
    # Ajouter des cercles invisibles pour améliorer la détection du survol
    fg_circles = p_fg.circle(x='timestamp', y='value', source=source_all, size=10, alpha=0, hover_alpha=0)
    
    # Nous ne dessinons plus les moyennes mobiles sur le graphique
    
    extreme_fear = BoxAnnotation(bottom=0, top=25, fill_alpha=0.15, fill_color='red')
    fear = BoxAnnotation(bottom=25, top=45, fill_alpha=0.15, fill_color='orange')
    neutral = BoxAnnotation(bottom=45, top=55, fill_alpha=0.15, fill_color='yellow')
    greed = BoxAnnotation(bottom=55, top=75, fill_alpha=0.15, fill_color='lightgreen')
    extreme_greed = BoxAnnotation(bottom=75, top=100, fill_alpha=0.15, fill_color='green')
    
    p_fg.add_layout(extreme_fear)
    p_fg.add_layout(fear)
    p_fg.add_layout(neutral)
    p_fg.add_layout(greed)
    p_fg.add_layout(extreme_greed)
    
    p_fg.xgrid.grid_line_color = None
    p_fg.ygrid.grid_line_color = None
    
    p_fg.xaxis.axis_line_color = "#555555"
    p_fg.yaxis.axis_line_color = "#555555"
    p_fg.xaxis.major_label_text_color = "#cccccc"
    p_fg.yaxis.major_label_text_color = "#cccccc"
    p_fg.title.text_color = "#ffffff"
    p_fg.legend.background_fill_color = "#1f1f1f"
    p_fg.legend.label_text_color = "#cccccc"
    p_fg.legend.border_line_color = None
    
    p_fg.y_range = Range1d(0, 100)
    
    # Créer un HoverTool qui fonctionne sur toute la zone du graphique
    hover_fg = HoverTool(
        tooltips="""
        <div style="background-color: rgba(42, 42, 42, 0.7); color: white; padding: 5px; border-radius: 3px; font-size: 11px;">
            <div>@timestamp{%F}</div>
            <div>Fear: @fear_percent{0.0}%</div>
        </div>
        """,
        formatters={'@timestamp': 'datetime'},
        mode='mouse',
        point_policy='snap_to_data',
        line_policy='nearest',
        renderers=[fg_line, fg_circles]
    )
    p_fg.add_tools(hover_fg)
    
    crosshair = CrosshairTool(line_color='#777777', line_alpha=0.5)
    p_ohlc.add_tools(crosshair)
    p_fg.add_tools(crosshair)
    
    plots = column(p_ohlc, p_fg, sizing_mode="stretch_width", spacing=0)
    
    return plots

def save_and_download_html(fig, filename):
    old_stdout = sys.stdout
    sys.stdout = io.StringIO()
    output_file(filename)
    save(fig)
    sys.stdout = old_stdout
    
    download_link = f"""
    <div style="margin-top: 20px; margin-bottom: 20px; padding: 10px; background-color: #2a2a2a; border-radius: 5px; color: white;">
        <p style="margin-bottom: 10px;"><strong>Télécharger le graphique :</strong></p>
        <a href="{filename}" download="{filename}" 
           style="background-color: #4CAF50; color: white; padding: 10px 15px; text-decoration: none; border-radius: 4px; display: inline-block;">
           Télécharger {filename}
        </a>
    </div>
    """
    display(HTML(download_link))

old_stdout = sys.stdout
sys.stdout = io.StringIO()
merged_df = get_data_silently()
sys.stdout = old_stdout

clear_output(wait=True)

if merged_df is not None:
    # Calculer les valeurs actuelles pour l'affichage
    current_value = merged_df['value'].iloc[-1]  # Valeur brute de l'indice
    current_ma31 = merged_df['ma31'].iloc[-1] if not pd.isna(merged_df['ma31'].iloc[-1]) else "N/A"
    current_ma62 = merged_df['ma62'].iloc[-1] if not pd.isna(merged_df['ma62'].iloc[-1]) else "N/A"
    current_ma90 = merged_df['ma90'].iloc[-1] if not pd.isna(merged_df['ma90'].iloc[-1]) else "N/A"
    current_ma365 = merged_df['ma365'].iloc[-1] if not pd.isna(merged_df['ma365'].iloc[-1]) else "N/A"
    current_date = merged_df['timestamp'].iloc[-1].strftime('%d/%m/%Y')
    
    # Formater les valeurs numériques
    if isinstance(current_ma31, float): current_ma31 = f"{current_ma31:.1f}"
    if isinstance(current_ma62, float): current_ma62 = f"{current_ma62:.1f}"
    if isinstance(current_ma90, float): current_ma90 = f"{current_ma90:.1f}"
    if isinstance(current_ma365, float): current_ma365 = f"{current_ma365:.1f}"
    
    print(f"""

## Indicateurs Fear & Greed (au {current_date})
- Fear & Greed: {current_value:.1f}
- MA31: {current_ma31}
- MA62: {current_ma62}
- MA90: {current_ma90}
- MA365: {current_ma365}
""")

    print("\nBitcoin et Fear & Greed - Période complète")
    fig_all = create_ohlc_fear_greed_bokeh(
        merged_df, 
        'Bitcoin (BTC/USDT) et Indice Fear & Greed - Période complète',
        period_name="(période complète)"
    )
    show(fig_all)
    save_and_download_html(fig_all, "Fear&GreedAll.html")
    
    print("\n## Bitcoin et Fear & Greed - Dernière année")
    last_year_data = merged_df.iloc[-365:].copy()
    fig_year = create_ohlc_fear_greed_bokeh(
        last_year_data, 
        'Bitcoin (BTC/USDT) et Indice Fear & Greed - Dernière année',
        period_name="(dernière année)"
    )
    show(fig_year)
    save_and_download_html(fig_year, "Fear&GreedYearly.html")
    
    print("""
    
    
    
    - Extreme Fear (0-25) : Souvent associé à des creux de marché
    - Fear (25-45) : Périodes d'incertitude
    - Neutral (45-55) : Sentiment équilibré
    - Greed (55-75) : Optimisme du marché
    - Extreme Greed (75-100): Potentiellement suracheté
    
    Vous pouvez télécharger les graphiques en HTML pour une analyse plus détaillée.
    """)



## Indicateurs Fear & Greed (au 06/03/2025)
- Fear & Greed: 25.0
- MA31: 40.4
- MA62: 55.4
- MA90: 61.6
- MA365: 61.2


Bitcoin et Fear & Greed - Période complète



## Bitcoin et Fear & Greed - Dernière année



    
    
    
    - Extreme Fear (0-25) : Souvent associé à des creux de marché
    - Fear (25-45) : Périodes d'incertitude
    - Neutral (45-55) : Sentiment équilibré
    - Greed (55-75) : Optimisme du marché
    - Extreme Greed (75-100): Potentiellement suracheté
    
    Vous pouvez télécharger les graphiques en HTML pour une analyse plus détaillée.
    
